In [34]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Carregar os dados
dsa = pd.read_csv("./data/respiratory_symptoms_and_treatment.csv")

# Preencher valores nulos e ajustar os dados conforme necessário (código que você já tem)
tratamento = dsa["Treatment"].value_counts().index[0]
dsa["Treatment"].fillna(tratamento, inplace=True)


def moda(coluna, index):
    moda = dsa[coluna].value_counts().index[index]
    return dsa[coluna].fillna(moda, inplace=True)


moda("Symptoms", 0)
moda("Age", 0)
moda("Sex", 0)
moda("Disease", 0)
moda("Treatment", 0)
moda("Nature", 0)

# Limpeza e ajustes nos dados (código que você já tem)
dsa["Symptoms"] = dsa["Symptoms"].replace(
    {
        " coughing ": "Cough",
        " shortness of breath": "Shortness of breath",
        " wheezing": "Wheezing",
        " low energy": "Low energy",
        " fatigue": "Fatigue",
        "whistling sound while breathing": "Whistling sound while you breathe",
        "Fatigue, feeling run-down or tired": "Fatigue",
        "Loss of appetite and unintentional weight loss": "Loss of appetite",
        "Rapid heartbeat\n": "Rapid heartbeat",
        "Shortness of breath that gets worse during flare-ups": "Shortness of breath",
        "Weight loss from loss of appetite": "Loss of appetite",
        "shortness of breath": "Shortness of breath",
        "short of breath": "Shortness of breath",
        "short, shallow and rapid breathing": "Rapid heartbeat",
        "allergy": "Allergy",
        "nausea": "Nausea",
        "loss of appetite": "Loss of appetite",
        "wheezing cough": "Wheezing",
        "high fever": "Fever",
        "fever": "Fever",
        "Persistent dry coug": "Cough",
        "chest pain": "Chest congestion",
        "coughing": "Cough",
        "cough": "Cough",
        "chronic cough": "Cough",
        "cough with blood": "Cough",
        "Morning headaches": "Headache",
        "pain": "Joint pain",
        "yellow cough": "Cough",
        "Coughing up blood": "Cough",
    }
)
dsa["Treatment"] = dsa["Treatment"].replace(
    {
        "Antibiotics.": "Antibiotic",
        "Antibiotics": "Antibiotic",
        "antibiotics": "Antibiotic",
    }
)

# Codificar colunas categóricas
symptoms_encoder = LabelEncoder()
sex_encoder = LabelEncoder()
nature_encoder = LabelEncoder()
treatment_encoder = LabelEncoder()

dsa["Symptoms"] = symptoms_encoder.fit_transform(dsa["Symptoms"])
dsa["Sex"] = sex_encoder.fit_transform(dsa["Sex"])
dsa["Nature"] = nature_encoder.fit_transform(dsa["Nature"])
dsa["Treatment"] = treatment_encoder.fit_transform(dsa["Treatment"])

# Preparar os dados para treinamento
X = dsa[["Symptoms", "Age", "Sex", "Nature"]]
y = dsa["Treatment"]

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Treinar o modelo RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Ajustar o modelo usando GridSearchCV
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}
grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


# Função para prever tratamentos com base nos sintomas fornecidos
def prever_tratamento(novos_sintomas, idade, sexo, natureza):
    # Verificar se os novos valores estão nos encoders
    if novos_sintomas not in symptoms_encoder.classes_:
        raise ValueError(f"Sintoma não reconhecido: {novos_sintomas}")
    if sexo not in sex_encoder.classes_:
        raise ValueError(f"Sexo não reconhecido: {sexo}")
    if natureza not in nature_encoder.classes_:
        raise ValueError(f"Natureza não reconhecida: {natureza}")

    # Preparar os dados de entrada
    novos_dados = pd.DataFrame(
        {
            "Symptoms": [novos_sintomas],
            "Age": [idade],
            "Sex": [sexo],
            "Nature": [natureza],
        }
    )

    # Codificar os dados de entrada
    novos_dados["Symptoms"] = symptoms_encoder.transform(novos_dados["Symptoms"])
    novos_dados["Sex"] = sex_encoder.transform(novos_dados["Sex"])
    novos_dados["Nature"] = nature_encoder.transform(novos_dados["Nature"])

    # Prever o tratamento
    predicao = best_model.predict(novos_dados)
    tratamento_previsto = treatment_encoder.inverse_transform(predicao)

    return tratamento_previsto[0]


# Exemplo de uso da função

Fitting 3 folds for each of 108 candidates, totalling 324 fits


ValueError: Sexo não reconhecido: Male

In [36]:
sintomas = "Cough"
idade = 25
sexo = "male"
natureza = "medium"

tratamento = prever_tratamento(sintomas, idade, sexo, natureza)
print(f"O tratamento previsto para os sintomas {sintomas} é: {tratamento}")

O tratamento previsto para os sintomas Cough é: Antibiotic
